<a href="https://colab.research.google.com/github/medha2716/image-seg-ship-detection/blob/main/sar_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Processing a 3200 * 3200 chunk

In [2]:
import skimage as ski
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from skimage.morphology import max_tree, label
import networkx as nx

def area_filtering(image, area_threshold):
    """Filter image based on area criteria."""
    labeled_image, num_features = label(image, connectivity=2, return_num=True)
    component_areas = np.bincount(labeled_image.ravel())

    for component_label in range(1, num_features + 1):
        if component_areas[component_label] <= area_threshold:
            image[labeled_image == component_label] = 0

    return image

def automatic_contrast_stretching(image, lower_percentile=5, upper_percentile=95):
    # Compute the lower and upper bounds using percentiles
    lower_bound = np.percentile(image, lower_percentile)
    upper_bound = np.percentile(image, upper_percentile)

    # Apply the contrast stretching formula
    stretched_image = np.clip((image - lower_bound) * 255 / (upper_bound - lower_bound), 0, 255)
    stretched_image = stretched_image.astype(np.uint16)

    return stretched_image # Add return statement to send processed image back

def process(chunk,chunk_name):

  chunk_contrast_stretching = automatic_contrast_stretching(chunk)
  # cv2.imwrite(os.path.join('contrast_stretched.jpg'), chunk_contrast_stretching)

  # blur before thresholding
  # high blurring so that larger landmasses are more connected
  blurred_image = chunk_contrast_stretching
  cv2.boxFilter(blurred_image, -1, (10,10), normalize=True, borderType=cv2.BORDER_CONSTANT)

  # perform automatic thresholding to get threshold for height
  t = ski.filters.threshold_otsu(blurred_image)
  # cv2.imwrite(os.path.join('blurred.jpg'), blurred_image)
  # print(t)

  # Step 2: Filter using Height Criteria
  height_threshold = t/2
  outheight = blurred_image >= height_threshold
  outheight2 = np.where( outheight, 255, 0).astype(np.uint8)
  # cv2.imwrite(os.path.join('outheight.jpg'), outheight2 )

  # Step 3: Filter using Area Criteria
  area_threshold = 3200 # Example threshold (approximate ship area)
  outarea = area_filtering(outheight, area_threshold)
  outarea = np.where( outarea, 255, 0).astype(np.uint8)
  # cv2.imwrite(os.path.join('outarea.jpg'), outarea)

  # Step 4: Subtract the filtered image
  result_image = outheight2 - outarea
  # cv2.imwrite(os.path.join('connected_components.jpg'), outheight2 - outarea)

  # Create a mask where resultant image has 255
  mask = (result_image == 255)

  # Apply the mask to retain original values from image where the mask is True
  resultant_image = np.where(mask, blurred_image, 0)
  threshold = resultant_image.max()
  # print(threshold)

  # cv2.imwrite(os.path.join('resultant.jpg'), resultant_image)
  threshold = resultant_image.max()/blurred_image.max()
  print( blurred_image.max())
  print("Threshold final for "+chunk_name+": ",threshold)

  print("Initial max and min of chunk: ",chunk.max(),chunk.min())
  chunk1 = (chunk - chunk.min()) / (chunk.max() - chunk.min())

  chunk1 = (chunk1 > threshold).astype(np.uint16)

  chunk1=np.ma.masked_array(chunk, mask=chunk1)
  print("Final max and min of chunk: ",chunk1.max(),chunk1.min())
  return chunk



# Further divide from 3200 * 3200 to 800 * 800 sz images

In [3]:
def divide_further(chunk, chunk_name, OUTPUT_DIR, window_size_final):
  row=0
  window_size = window_size_final
  dataraster = chunk
  xSize = 3200
  ySize = 3200 # (size of the larger chunk)
  while row < ySize:
          col=0
          while col < xSize:
            if row + window_size <= ySize and col + window_size <= xSize:
                  data_value = dataraster[row:row + window_size, col:col + window_size]
                  cv2.imwrite(os.path.join(OUTPUT_DIR + chunk_name + '_' + str(row) + '_' + str(col) + '.jpg'), data_value)
            else:
                  pass
            col=col+window_size
          row=row+window_size

# Starts processing of dataraster

In [4]:
import cv2
import numpy as np
from scipy.stats import gaussian_kde
from scipy import stats as st
import matplotlib.pyplot as plt
import os
import math

def processRaster(dataraster,OUTPUT_DIR_PROCESSED,OUTPUT_DIR_DIRECT,window_size_final,xSize,ySize):

  '''
  divides raster into 3200*3200 chunks
  processes each chunk
  further divides them into the desired window size
  '''


  global_max=int(dataraster.max())
  global_min=int(dataraster.min())
  print("Max and min values of the dataraster: ",global_max,global_min)
  print("Shape of dataraster: ", dataraster.shape)

  window_size = 3200
  row=0
  while row < ySize:
          print("Row="+str(row))
          col=0
          while col < xSize:
            if row + window_size <= ySize and col + window_size <= xSize:
                  subRaster = dataraster[row:row + window_size, col:col + window_size]

                  data_value = numpy.zeros((3, window_size, window_size)).astype("uint16")
                  data_value[0, :, :] = subRaster
                  data_value[1, :, :] = subRaster
                  data_value[2, :, :] = subRaster

                  data_value = np.transpose(data_value, (1, 2, 0))

                  chunk_name = 'chunk' + str(row) + '_' + str(col)
                  processed_chunk = process(data_value,chunk_name)
                  divide_further(processed_chunk, chunk_name, OUTPUT_DIR_PROCESSED, window_size_final)
                  divide_further(data_value, chunk_name, OUTPUT_DIR_DIRECT, window_size_final) # directly divide without further processing

            else:
                  pass
            col=col+window_size
          row=row+window_size
  print("Done")


# Read the input Tiff File

In [5]:
from osgeo import gdal, ogr, osr
import os
gdal.UseExceptions()

import pandas as pd
import numpy
import cv2
from sklearn.preprocessing import StandardScaler,MinMaxScaler

def readRaster(INPUT_FILE,OUTPUT_DIR_PROCESSED,OUTPUT_DIR_DIRECT,window_size_final):
  '''
  read the tiff file and puts it into a numpy array
  '''
  print(INPUT_FILE)
  raster = gdal.Open(INPUT_FILE);
  # Get raster georeference info
  transform = raster.GetGeoTransform()
  xOrigin = transform[0]
  yOrigin = transform[3]
  pixelWidth = transform[1]
  pixelHeight = transform[5]
  xSize = raster.RasterXSize
  ySize = raster.RasterYSize
  print(transform)
  # Read raster data as a NumPy array
  print(xOrigin,yOrigin,xSize,ySize,pixelWidth,pixelHeight)
  dataraster = raster.ReadAsArray(0, 0, xSize, ySize).astype(numpy.uint16)
  processRaster(dataraster,OUTPUT_DIR_PROCESSED,OUTPUT_DIR_DIRECT,window_size_final,xSize,ySize)




# Input file, Output Dir defined

In [6]:
import os

if __name__ == "__main__":

    window_size_final=800
    INPUT_FILE='/content/drive/My Drive/S1A_IW_GRDH_1SDV_20240323T152959_20240323T153028_053111_066EE3_3DFF.SAFE/measurement/s1a-iw-grd-vh-20240323t152959-20240323t153028-053111-066ee3-002.tiff'
    OUTPUT_DIR_PROCESSED='/content/drive/My Drive/processed_divided_image_chunks/'
    OUTPUT_DIR_DIRECT = '/content/drive/My Drive/divided_image_chunks/'

    if not os.path.exists(OUTPUT_DIR_PROCESSED):
        os.makedirs(OUTPUT_DIR_PROCESSED)

    if not os.path.exists(OUTPUT_DIR_DIRECT):
        os.makedirs(OUTPUT_DIR_DIRECT)

    readRaster(INPUT_FILE,OUTPUT_DIR_PROCESSED,OUTPUT_DIR_DIRECT,window_size_final)

/content/drive/My Drive/S1A_IW_GRDH_1SDV_20240323T152959_20240323T153028_053111_066EE3_3DFF.SAFE/measurement/s1a-iw-grd-vh-20240323t152959-20240323t153028-053111-066ee3-002.tiff
(0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
0.0 0.0 25191 19442 1.0 1.0
Max and min values of the dataraster:  13765 0
Shape of dataraster:  (19442, 25191)
Row=0


/usr/local/lib/python3.10/dist-packages/skimage/_shared/utils.py:348: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  return func(*args, **kwargs)
<ipython-input-2-d3194c0aadfa>:40: UserWarning: threshold_otsu is expected to work correctly only for grayscale images; image shape (3200, 3200, 3) looks like that of an RGB image.
  t = ski.filters.threshold_otsu(blurred_image)


45343.61812476225
Threshold final for chunk0_0:  0.022097316464068442
Initial max and min of chunk:  1559 0
Final max and min of chunk:  34 0
Row=0
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
Row=800
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
Row=1600
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
Row=2400
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
Row=0
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
Row=800
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
Row=1600
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
Row=2400
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
36137.862910473006
Threshold final for chunk0_3200:  0.4477799371883118
Initial max and min of chunk:  100 0
Final max and min of chunk:  44 0
Row=0
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
Row=800
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
Row=1600
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
(800, 800, 3)
Row=2400
(80

KeyboardInterrupt: 